## Реализация StyleGAN-NADA

Клонируем эталонную реализацию StyleGAN2 для PyTorch

In [ ]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch

In [ ]:
import os

import clip
import dnnlib
import pretrained_networks
import torch
import torch.nn.functional as F
from training.networks import Generator

#### Загрузка моделей

In [ ]:
clip_model, clip_preprocess = clip.load("ViT-B/32", device="cuda")
stylegan_network = pretrained_networks.load_network("path_to_stylegan_pkl")

In [ ]:
# Допустим, у вас клонирован репозиторий StyleGAN2-ADA (https://github.com/NVlabs/stylegan2-ada-pytorch)
# В нем есть модули, отвечающие за тренинг, загрузку сетей, расширения данных и т. д.
# Ниже - условные внутрипроектные импорты (примерные названия, в реальном коде могут отличаться):


# -------------------------------------------------------------------
# Основные идеи:
# 1. У нас есть два текстовых описания: исходный домен (src_text) и целевой (tgt_text).
# 2. Вычисляем delta = E(tgt_text) - E(src_text) в пространстве CLIP.
# 3. Для батча сэмплов (или нескольких случайных z) получаем сгенерированные изображения,
#    вычисляем их эмбеддинги E(I_g), и двигаем их к E(I_g) + delta (directional loss).
# 4. Замораживаем ранние слои, чтобы «сохранить» глобальную структуру, а стили меняем на более поздних слоях.
# -------------------------------------------------------------------


def freeze_layers(gen: Generator, num_freeze_layers=2):
    """
    Пример: частично замораживаем слои генератора.
    Допустим, что gen.synthesis состоит из списка блоков, где первые num_freeze_layers
    оставляем без обучения (requires_grad=False).
    """
    blocks = list(gen.synthesis.children())
    for i, block in enumerate(blocks):
        if i < num_freeze_layers:
            for param in block.parameters():
                param.requires_grad = False


def compute_directional_loss(clip_model, clip_preprocess, images, src_emb, delta_t, alpha=1.0):
    """
    Суть StyleGAN-NADA: двигаем эмбеддинги генерируемых изображений E(I_g)
    в направлении delta_t = E(tgt_text) - E(src_text).

    Псевдо-формула:
       Loss = 1 - cos( E(I_g) - src_emb, delta_t )
    Также могут использоваться разные вариации directional loss,
    см. оригинальную статью StyleGAN-NADA.
    """
    # Приводим изображения к формату (B, C, H, W), желательно 224x224
    # Если необходимо, делаем resize
    images_224 = torch.nn.functional.interpolate(images, size=(224, 224), mode="bilinear")
    # Преобразуем под CLIP
    imgs_clip_ready = clip_preprocess(images_224)
    E_ig = clip_model.encode_image(imgs_clip_ready)  # [B, dim]

    # Разница E(I_g) - E(src_text)
    diff = E_ig - src_emb
    # Косинусная близость между diff и delta_t
    cos_sim = F.cosine_similarity(diff, delta_t.unsqueeze(0), dim=1)
    # Превращаем в loss (хотим максимизировать косинусную близость => минимизируем -cos_sim)
    directional_loss = alpha * (1 - cos_sim.mean())

    return directional_loss


def train_nada(
    network_pkl,  # Исходный путь к уже обученной StyleGAN2-ADA модели
    src_text,  # Текст, описывающий исходный домен
    tgt_text,  # Текст, описывающий целевой стиль/домен
    outdir="nada-out",  # Папка для результатов
    num_steps=1000,
    batch_size=4,
    lr=0.002,
    freeze_layers_num=2,
):
    """
    Дообучение StyleGAN2-ADA в стиле NADA:
    объединяем официальный код StyleGAN2-ADA с механизмом directional CLIP loss.
    """
    device = torch.device("cuda")

    # 1. Загрузка исходной модели
    with dnnlib.util.open_url(network_pkl) as f:
        G = torch.load(f)["G_ema"].to(device)  # Часто сеть хранится в словаре

    # 2. Инициализация CLIP
    clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)
    clip_model.eval()

    # 3. Получаем эмбеддинги для src_text и tgt_text
    token_src = clip.tokenize([src_text]).to(device)
    token_tgt = clip.tokenize([tgt_text]).to(device)
    with torch.no_grad():
        emb_src = clip_model.encode_text(token_src)[0]  # shape [dim]
        emb_tgt = clip_model.encode_text(token_tgt)[0]  # shape [dim]
    delta_t = emb_tgt - emb_src  # Вектор смещения

    # 4. Замораживаем нужные слои
    freeze_layers(G, num_freeze_layers=freeze_layers_num)

    # 5. Оптимизируем только те параметры, что не заморожены
    params = [p for p in G.parameters() if p.requires_grad]
    optimizer = torch.optim.Adam(params, lr=lr)

    # 6. Основной цикл обучения
    for step in range(num_steps):
        optimizer.zero_grad()

        # Сэмплируем случайный шум
        z = torch.randn(batch_size, G.z_dim, device=device)
        # Прогоняем через мэппинг
        ws = G.mapping(z, None)  # [batch, num_ws, w_dim]

        # Генерация изображений
        synth_images = G.synthesis(ws, noise_mode="const")  # [B, 3, H, W]

        # Вычисляем Directional CLIP loss
        loss = compute_directional_loss(clip_model, clip_preprocess, synth_images, emb_src, delta_t)

        # Добавляем свою регуляризацию или ADA-augmentations (см. офиц. реализацию)
        # Например, можно смешать loss с оригинальным прогнозом дискриминатора,
        # но в простейшем варианте оставим чистый directional loss
        loss.backward()
        optimizer.step()

        if step % 50 == 0:
            print(f"Step {step}/{num_steps}   Loss: {loss.item():.4f}")

            # Пример сохранения одного изображения
            with torch.no_grad():
                out_z = torch.randn(1, G.z_dim, device=device)
                out_ws = G.mapping(out_z, None)
                out_img = G.synthesis(out_ws, noise_mode="const")[0]
                out_img_np = (
                    (out_img.permute(1, 2, 0).cpu().numpy() * 127.5 + 127.5)
                    .clip(0, 255)
                    .astype("uint8")
                )
                pil_img = Image.fromarray(out_img_np)

                os.makedirs(outdir, exist_ok=True)
                pil_img.save(f"{outdir}/step_{step:04d}.png")

    # 7. Сохранение итоговых весов
    os.makedirs(outdir, exist_ok=True)
    final_pkl = os.path.join(outdir, "stylegan2_nada_final.pkl")
    torch.save({"G_ema": G.state_dict()}, final_pkl)
    print(f"Done! Model saved to {final_pkl}")


# -------------------------------------------------------------------
# Пример вызова:
# python train_nada.py --network_pkl=/path/to/original.pkl --src_text="photo" --tgt_text="a cubist painting"
# (с последующей интеграцией в ваш пайплайн StyleGAN2-ADA)
# -------------------------------------------------------------------